# Label CT Scan cracks

This notebook is used to label CT Scan crack

## Open Image and (possibly) pre-existing label

In [1]:
from skimage.io import imread, imsave
import napari
import os
import numpy as np
from pathlib import Path
from tnia.deeplearning.dl_helper import make_label_directory

tnia_images_path = Path(r'D:/images')
parent_path=Path(tnia_images_path / r'tnia-python-images/imagesc/2024_05_07_ct_scan')
label_path = Path(parent_path / r'labels')
image_path = Path(parent_path / r'images')
name = 'Cored_NonMachined_rec00000489'
ext = '.bmp'

num_classes = 2

image_label_paths, mask_label_paths = make_label_directory(1, 2, label_path)

for mask_label_name in image_label_paths:
    print(mask_label_name)
print()

im = imread(os.path.join(image_path / (name+ext)))

# this image is really gray scale with each channel containing the same info so keep only the first channel
im = im[:,:,0]
labels = []

try:
    for mask_label_path in mask_label_paths:

        for i in range(num_classes):
            label = imread(os.path.join(mask_label_path / (name+".tif")))
            labels.append(label)
except:
    print('labels not created yet', (os.path.join(label_path / (name+".tif"))))
    for i in range(num_classes):
        labels.append(np.zeros([im.shape[0], im.shape[1]], dtype=np.uint8))

print('there are', len(labels), 'labels')

D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\input0

labels not created yet D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\Cored_NonMachined_rec00000489.tif
there are 2 labels


## Edit an ROI of the labels in Napari

Now edit the labels in Napari.  We can start from a blank image or alternatively a pre-existing set of labels (either generated by another method, or previous manual labels) that we want to improve.

Use the box ROI to indicate which region of the image contains labels.

In [2]:
viewer = napari.Viewer()
viewer.add_image(im, name='im')
for i in range(num_classes):
    viewer.add_labels(labels[i], name='labels'+str(i))

boxes_layer = viewer.add_shapes(
            name="Label box",
            face_color="transparent",
            edge_color="green",
            edge_width=2,
        )

boxes_layer.add([[[0,0],[im.shape[0],im.shape[1]//2]]])

In [10]:
im = viewer.layers['im'].data
im.shape

for i in range(num_classes):
    labels[i] = viewer.layers['labels'+str(i)].data

labels[0].shape

(1944, 1944)

## Get the ROI that was labeled

In [11]:
ystart = int(np.min(boxes_layer.data[0][:,0]))
yend = int(np.max(boxes_layer.data[0][:,0]))
xstart = int(np.min(boxes_layer.data[0][:,1]))
xend = int(np.max(boxes_layer.data[0][:,1]))

print('bounding box is',ystart, yend, xstart, xend)
if np.ndim(im) == 3:
    im = im[ystart:yend, xstart:xend, :]
else:
    im = im[ystart:yend, xstart:xend]

for i in range(num_classes):
    labels[i] = labels[i][ystart:yend, xstart:xend]

bounding box is 527 1203 83 1055


In [16]:
im.shape

(676, 972)

## Save modified labels

In [13]:
from tnia.deeplearning.dl_helper import generate_patch_names, generate_next_patch_name

image_name, mask_name = generate_patch_names(str(image_label_paths[0]), str(mask_label_path), name)

base_name = generate_next_patch_name(str(image_label_paths[0]), name)

print(base_name)

print(image_name)
print(mask_name)
print(base_name)


Cored_NonMachined_rec00000489_1
D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\input0/Cored_NonMachined_rec00000489_1.tif
D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\ground truth0/Cored_NonMachined_rec00000489_1.tif
Cored_NonMachined_rec00000489_1


In [14]:
imsave(image_name, im)
print(image_name)

for i in range(num_classes):
    print(mask_label_paths[i])
    imsave(os.path.join(mask_label_paths[i], base_name+".tif"), labels[i])
#imsave(mask_name, labels)
#boxes_layer.save(csv_name)

D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\input0/Cored_NonMachined_rec00000489_1.tif
D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\ground truth0
D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\ground truth1


C:\Users\bnort\AppData\Local\Temp\ipykernel_30360\148810999.py:6: UserWarning: D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\ground truth0\Cored_NonMachined_rec00000489_1.tif is a low contrast image
  imsave(os.path.join(mask_label_paths[i], base_name+".tif"), labels[i])
C:\Users\bnort\AppData\Local\Temp\ipykernel_30360\148810999.py:6: UserWarning: D:\images\tnia-python-images\imagesc\2024_05_07_ct_scan\labels\ground truth1\Cored_NonMachined_rec00000489_1.tif is a low contrast image
  imsave(os.path.join(mask_label_paths[i], base_name+".tif"), labels[i])


In [15]:
# save xstart, ystart, xend, yend to json 
import json
json_name = os.path.join(image_label_paths[0], base_name+".json")
with open(json_name, 'w') as f:
    json_ = {}
    json_['base_name'] = base_name
    json_['bbox'] = [xstart, ystart, xend, yend]
    json.dump(json_, f)